<a href="https://colab.research.google.com/github/Fulim13/Storyteller/blob/test/api/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Data Preprocessing


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

In [3]:

# Load dataset
# Replace with your dataset name
dataset = load_dataset('FareedKhan/1k_stories_100_genre')

# Load NER pipeline
model_name = "dslim/bert-base-NER"  # Pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline(
    "ner", model=model, tokenizer=tokenizer, grouped_entities=True, device=0)

# Function to extract characters using NER


def extract_characters(example):
    story = example['story']
    entities = ner_pipeline(story)  # Run NER on the story
   # Filter out only PERSON entities and ensure uniqueness
    characters = list(
        set(entity['word'] for entity in entities if entity['entity_group'] == 'PER'))
    return {"characters": characters}


# Apply the function to add the characters column
dataset = dataset.map(extract_characters)

# Inspect the updated dataset
print(dataset)

README.md:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

1k_stories_100_genre.csv:   0%|          | 0.00/5.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'story', 'genre', 'characters'],
        num_rows: 1000
    })
})


In [4]:
dataset['train']['characters'][:5]

[['Art',
  'Reynolds',
  'Arthur',
  'Evelyn',
  'Simmons',
  'Amelia Hart',
  'Eve',
  'Jack',
  'Hart',
  'Turner',
  'John'],
 ['O', 'Thorn'],
 ['Johnath', 'John'],
 ['William', 'Thomas', 'Elias', 'Blackwood'],
 ['Katie', 'Mark', 'Sarah', 'Ben', 'Alex']]

In [17]:
def format_prompt(example):
    """Create a structured and guiding prompt for story generation."""

    # Extract data from the example
    genre = example["genre"]
    # Join characters into a single string
    characters = ", ".join(example["characters"])
    story = example["story"]
    title = example["title"]

    # Create the prompt text with a guiding introduction
    prompt = (
        "Create a story based on the given genre and title. "
        f"Genre: {genre}\n"
        f"Title:\n{title}\n\n"
        "Ensure the output includes the following in order:\n"
        "1. Characters\n"
        "2. The story\n"
        "Output:\n"
        f"Characters: {characters}\n"
        f"Story: {story}\n"
    )

    return {"text": prompt}


# Apply the formatting to the dataset
formatted_dataset = dataset.map(format_prompt)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [6]:
print(formatted_dataset['train']['text'][:5])

['Create a story based on the given genre and title. Genre: Science Fiction\nTitle:\nThe Chronicles of the Cosmic Rift\n\nEnsure the output includes the following in order:\n1. Characters\n2. The story\nOutput:\nCharacters: Art, Reynolds, Arthur, Evelyn, Simmons, Amelia Hart, Eve, Jack, Hart, Turner, John\nStory: In the year 2250, Earth had made significant strides in space exploration and interstellar travel. The United Earth Government (UEG) had established colonies on Mars, Jupiter\'s moon Europa, and Saturn\'s moon Titan. The advancements in technology and science had led to the creation of the Cosmic Rift Exploration Agency (CREA), a government-funded organization tasked with exploring the unknown regions of space and discovering new worlds and resources.\n\n    Dr. Amelia Hart, a brilliant astrophysicist, was the lead scientist at CREA\'s headquarters on Luna. She had devoted her entire life to understanding the mysteries of the universe and had become a pioneer in her field. She

In [ ]:
formatted_dataset.save_to_disk("./datasets/processed_stories")

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 18200.65 examples/s]


In [ ]:
from datasets import load_from_disk

# Load the dataset from the saved directory
# Replace with the path you used earlier
dataset = load_from_disk("./datasets/processed_stories")

# Inspect the loaded dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'story', 'genre', 'characters', 'text'],
        num_rows: 1000
    })
})


In [18]:
dataset = formatted_dataset.filter(lambda example, index: index %
                         100 == 0, with_indices=True)
print(dataset)

Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'story', 'genre', 'characters', 'text'],
        num_rows: 1
    })
})


In [50]:
from google.colab import userdata
HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN')

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.1-8B"

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
    llm_int8_enable_fp32_cpu_offload=True,
)

# Load the model to train on the GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # Leave this out for regular SFT
    quantization_config=bnb_config,
    use_auth_token=HUGGINGFACE_TOKEN
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [11]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA Configuration
# peft_config = LoraConfig(
#     lora_alpha=32,  # LoRA Scaling
#     lora_dropout=0.1,  # Dropout for LoRA Layers
#     r=64,  # Rank
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=[
#         # Self-attention projection layers
#         'self_attn.q_proj',
#         'self_attn.k_proj',
#         'self_attn.v_proj',
#         'self_attn.o_proj',

#         # MLP layers
#         'mlp.gate_proj',
#         'mlp.up_proj',
#         'mlp.down_proj'
#     ]
# )

peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=# Layers to target
    ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [12]:
model.get_nb_trainable_parameters()

(167772160, 8198033408)

In [13]:
from transformers import TrainingArguments

output_dir = "./results"

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

In [14]:
for name, module in model.named_modules():
    print(name)


base_model
base_model.model
base_model.model.model
base_model.model.model.embed_tokens
base_model.model.model.layers
base_model.model.model.layers.0
base_model.model.model.layers.0.self_attn
base_model.model.model.layers.0.self_attn.q_proj
base_model.model.model.layers.0.self_attn.q_proj.base_layer
base_model.model.model.layers.0.self_attn.q_proj.lora_dropout
base_model.model.model.layers.0.self_attn.q_proj.lora_dropout.default
base_model.model.model.layers.0.self_attn.q_proj.lora_A
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default
base_model.model.model.layers.0.self_attn.q_proj.lora_B
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default
base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_A
base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_B
base_model.model.model.layers.0.self_attn.q_proj.lora_magnitude_vector
base_model.model.model.layers.0.self_attn.k_proj
base_model.model.model.layers.0.self_attn.k_proj.base_layer
base_model.mode

In [29]:
print(dataset["train"][0])  # Inspect the first example in the dataset

{'id': 457580, 'title': 'The Chronicles of the Cosmic Rift', 'story': 'In the year 2250, Earth had made significant strides in space exploration and interstellar travel. The United Earth Government (UEG) had established colonies on Mars, Jupiter\'s moon Europa, and Saturn\'s moon Titan. The advancements in technology and science had led to the creation of the Cosmic Rift Exploration Agency (CREA), a government-funded organization tasked with exploring the unknown regions of space and discovering new worlds and resources.\n\n    Dr. Amelia Hart, a brilliant astrophysicist, was the lead scientist at CREA\'s headquarters on Luna. She had devoted her entire life to understanding the mysteries of the universe and had become a pioneer in her field. She was determined to uncover the secrets of the cosmic rifts, a series of mysterious and seemingly unconnected energy anomalies that had started appearing throughout the galaxy.\n\n    Dr. Hart assembled a diverse team of experts for her next mis

In [33]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(
        example["story"],  # Use the 'story' field for tokenization
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Verify tokenized dataset
print(tokenized_dataset["train"][0])  # Check the tokenized output

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

{'id': 457580, 'title': 'The Chronicles of the Cosmic Rift', 'story': 'In the year 2250, Earth had made significant strides in space exploration and interstellar travel. The United Earth Government (UEG) had established colonies on Mars, Jupiter\'s moon Europa, and Saturn\'s moon Titan. The advancements in technology and science had led to the creation of the Cosmic Rift Exploration Agency (CREA), a government-funded organization tasked with exploring the unknown regions of space and discovering new worlds and resources.\n\n    Dr. Amelia Hart, a brilliant astrophysicist, was the lead scientist at CREA\'s headquarters on Luna. She had devoted her entire life to understanding the mysteries of the universe and had become a pioneer in her field. She was determined to uncover the secrets of the cosmic rifts, a series of mysterious and seemingly unconnected energy anomalies that had started appearing throughout the galaxy.\n\n    Dr. Hart assembled a diverse team of experts for her next mis

In [36]:
from trl import SFTTrainer
import os
from google.colab import output

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,

    # Leave this out for regular SFT
    peft_config=peft_config,
)

# Train model
trainer.train()



# Attempt to save the model
try:
    # Ensure the directory exists
    os.makedirs("Llama3-8B-Instruct-v0.3-QLoRA", exist_ok=True)

    # Save the model
    trainer.model.save_pretrained("Llama3-8B-Instruct-v0.3-QLoRA")

    # Save the tokenizer as well
    tokenizer.save_pretrained("Llama3-8B-Instruct-v0.3-QLoRA")

    # Verify the save
    print("Contents of saved model directory:")
    print(os.listdir("Llama3-8B-Instruct-v0.3-QLoRA"))

except Exception as e:
    print(f"Error saving model: {e}")
    output.eval_js('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Step,Training Loss


Contents of saved model directory:
['tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json', 'adapter_config.json', 'README.md', 'adapter_model.safetensors']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save to Google Drive
save_path = "/content/drive/My Drive/Llama3-8B-Instruct-v0.3-QLoRA"
os.makedirs(save_path, exist_ok=True)

try:
    # Save model to Drive
    trainer.model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"Model saved to {save_path}")
    print("Contents of saved model directory:")
    print(os.listdir(save_path))

except Exception as e:
    print(f"Error saving model to Drive: {e}")

In [1]:
from peft import AutoPeftModelForCausalLM
from accelerate import init_empty_weights, dispatch_model, disk_offload

# Initialize weights on disk to avoid out-of-memory errors
with init_empty_weights():
    model = AutoPeftModelForCausalLM.from_pretrained(
        "Llama3-8B-Instruct-v0.3-QLoRA",
        low_cpu_mem_usage=True,
    )

# Offload model to disk
disk_offload(model, "./offload", device="cuda:0")

# Merge LoRA and base model
merged_model = model.merge_and_unload()


ValueError: Can't find 'adapter_config.json' at 'Llama3-8B-Instruct-v0.3-QLoRA'

In [ ]:
# Save the merged model
merged_model.save_pretrained("Mistral-7B-Instruct-v0.3-Merged")
tokenizer.save_pretrained("Mistral-7B-Instruct-v0.3-Merged")

In [ ]:
from transformers import pipeline

# Use our predefined prompt template
prompt = """Create a story based on the given genre and title.
"""

# Run our instruction-tuned model
pipe = pipeline(task="text-generation",
                model=merged_model, tokenizer=tokenizer)
print(pipe(prompt))

In [ ]:
prompt = """Create a story based on the given genre and title.
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move to GPU
outputs = merged_model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))